In [205]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [206]:
from src.data.utils import read_embeddings_dataframe

In [207]:
df = read_embeddings_dataframe("embeddings.csv")

In [208]:
type(df['embedding'].iloc[0])

numpy.ndarray

In [209]:
from src.models.clustering import perform_clusterings, create_clustering_files

In [210]:
results = perform_clusterings(df, 5, 5, 0.8)
create_clustering_files(results, ".", "clusters", "cluster_models")

In [211]:
from src.models.dimensionality_reduction import create_umap_data_and_model
create_umap_data_and_model(df, ".", "graph_umap", "umap")

In [203]:
import plotly.express as px
import pandas as pd
umap_df = pd.read_csv("graph_umap.csv")
cluster_info = pd.read_csv("clusters.csv")
whole_df = pd.merge(umap_df, cluster_info, on='username')
whole_df['kmeans_cluster'] = whole_df['kmeans_cluster'].astype(str)
whole_df['gmm_cluster'] = whole_df['gmm_cluster'].astype(str)
whole_df['mean_shift_cluster'] = whole_df['mean_shift_cluster'].astype(str)

fig = px.scatter(whole_df, x='2D_x', y='2D_y', color='mean_shift_cluster', hover_data=['username'])
fig.show(renderer='browser')

In [204]:
fig = px.scatter_3d(whole_df, x='3D_x', y='3D_z', z='3D_y', color='mean_shift_cluster', hover_data=['username'])
fig.show(renderer='browser')

In [119]:
from src.models.clustering import cluster_db_scan
data = whole_df[['3D_x', '3D_y', '3D_z']].to_numpy()
model, labels = cluster_db_scan(data, 0.4)
whole_df['super_dbscan'] = labels
whole_df['super_dbscan'] = whole_df['super_dbscan'].astype(str)
fig = px.scatter_3d(whole_df, x='3D_x', y='3D_y', z='3D_z', color='super_dbscan', hover_data=['username'])
fig.show(renderer='browser')

In [124]:
from src.models.dimensionality_reduction import perform_umap
import numpy as np
data = np.array(df["embedding"].tolist())
umap_10_model, umap_10_reduced = perform_umap(data, 10)

In [131]:
_, umap_10_dbscan = cluster_db_scan(umap_10_reduced, 0.3)
whole_df['umap_10'] = umap_10_dbscan
whole_df['umap_10'] = whole_df['umap_10'].astype(str)
fig = px.scatter_3d(whole_df, x='3D_x', y='3D_y', z='3D_z', color='umap_10', hover_data=['username'])
fig.show(renderer='browser')

In [133]:
data.shape

(548, 768)

In [134]:
from sklearn.metrics.pairwise import euclidean_distances 

distances = euclidean_distances(data)

In [140]:
distances = np.reshape(distances, -1)
distances.shape

(300304,)

In [141]:
fig = px.histogram(distances)
fig.show(renderer='browser')